## Gait Video Study 
### Identifying frames with HSRs in each video for each cohort and trial to establish break points and also evaluate the corresponding HSR labelling via the ground truth available. Further, downsample with smoothing to define fixed shape of the input tensor for models. 
#### Remember to preserve the original count of frames in a single stride (before down sampling via smoothing) for each stride to add as an additional artificial feature later to add information about speed of the subject to the model

In [1]:
import numpy as np
import cv2
import os
import glob
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import time
import shutil
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display, HTML

In [2]:
path = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data'
frame_path_merged = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\multi_view_merged_data\\'

#Configuration for which to run the code for 
cohorts = ['\\HOA', '\\MS', '\\PD', '\\ExtraHOA']
trials = ['\\beam_walking', '\\walking']
cameras = ['\\feet\\', '\\lower_body\\']

order = ['right hip', 'right knee', 'right ankle', 'left hip', 'left knee', 'left ankle', 'left toe 1', 'left toe 2', \
         'left heel', 'right toe 1', 'right toe 2', 'right heel']

### Utility functions 

In [5]:
#Saving the HSRframes.txt file to the hip_height_normalized\\ containing the final .csvs for analysis
for cohort in cohorts:
    for trial in trials:
        merged_path = frame_path_merged+cohort+trial 
        if (os.path.exists(merged_path)):
            videos = os.listdir(merged_path)
#             print (len(videos))
        for video in videos:
            print (glob.glob(path+cohort+trial+'\\feet\\'+'Inked'+video+'_0_Trim'))
            try:
                if (not os.path.exists(merged_path+'\\'+video+'\\HSRframes.txt')):
                    HSR_frames_file = path+cohort+trial+'\\feet\\'+'Inked'+video+'_0_Trim'+'\\HSRframes.txt'
                    shutil.copy(HSR_frames_file, merged_path+'\\'+video+'\\hip_height_normalized\\') 
                    print ('HSR for', video, 'copied')
                else:
                    print ('HSR for', video, 'exists')
            except Exception as e:
                print (e)

[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_212_T_T1_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_212_T_T2_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_213_T_T1_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_213_T_T2_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_214_T_T1_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Proje

In [3]:
labels = ['frame_number'] + [o + '-'+ y for o in order for y in ['x', 'y', 'z']]

for cohort in cohorts[1:2]:
    for trial in trials[:1]:
        merged_path = frame_path_merged+cohort+trial 
        if (os.path.exists(merged_path)):
            videos = os.listdir(merged_path)
#             print (len(videos))
        for video in videos[2:3]:
            trueHSR = open(merged_path+'\\'+video+'\\hip_height_normalized\\'+'\\HSRframes.txt').read()
            print (trueHSR)
            frames = glob.glob(merged_path+'\\'+video+'\\hip_height_normalized\\*.csv')
            #First, we need to sort the frames 
            sorted_frames = sorted(frames,  key=lambda name: int(name.split('\\')[-1][:-4]))
            temp = pd.DataFrame(columns = labels)
            for frame in sorted_frames:
                #Append the frame number also in the temp dataframe, since it the true HSR is given in frame number
                frame_no = int(frame.split('\\')[-1][:-4])
                frame_csv = pd.read_csv(frame, index_col = 0)
                temp.loc[len(temp)] = np.append(frame_no, frame_csv[['x', 'y', 'z']].values.flatten())
            temp = temp.astype({'frame_number': 'int'})
            temp.set_index('frame_number', inplace = True)
            display(temp)

35, 71, 107, 143, 179, 214, 250, 287, 320, 356, 392, 427, 462, 500, 536, 570, 606, 641, 674, 706, 737, 769, 800, 829, 860, 889, 917, 947, 977


,right hip-x,right hip-y,right hip-z,right knee-x,right knee-y,right knee-z,right ankle-x,right ankle-y,right ankle-z,left hip-x,...,left heel-z,right toe 1-x,right toe 1-y,right toe 1-z,right toe 2-x,right toe 2-y,right toe 2-z,right heel-x,right heel-y,right heel-z
frame_number,,,,,,,,,,,,,,,,,,,,,
0,39.449136,162.131598,100.184257,36.522369,143.974399,87.395076,34.058394,104.701142,26.561741,20.414385,...,26.447676,34.734083,148.392273,22.461395,38.667728,143.412639,21.829892,32.019455,139.504274,28.874307
1,39.453248,162.145605,100.192216,36.525768,143.985215,87.403087,34.926049,104.691449,26.578776,19.565527,...,26.439908,34.495741,116.735102,18.592852,38.969747,115.313934,19.548701,33.300731,108.728464,23.897386
2,39.777655,164.810217,99.938657,36.855551,146.956253,86.765595,35.172231,105.483416,26.752898,20.601070,...,25.791256,31.941744,116.303816,19.569412,37.405932,116.174572,19.671773,34.452235,109.563609,24.045646
3,40.576205,164.318651,99.697618,36.720703,144.624022,87.810100,35.076347,105.204018,26.674837,21.426205,...,23.986813,32.810106,117.280771,18.661366,38.263885,117.211643,18.723840,34.406710,110.549174,23.129102
4,40.814521,165.856493,99.703791,37.859332,146.285743,87.614330,35.283373,105.706628,26.815713,20.673328,...,26.676634,34.783622,117.879816,18.743699,39.390810,117.722684,18.861933,34.491380,109.765016,24.122696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,33.257117,130.605967,100.000000,30.563866,147.224373,88.762443,31.441949,155.152298,13.520264,7.069172,...,41.828280,31.471683,175.545710,0.000083,35.720899,172.229849,2.211375,30.152580,159.485252,10.643699
985,33.964744,130.588627,100.000000,30.616590,148.772599,85.665044,29.207571,153.064447,12.826504,7.072159,...,39.712779,29.902723,172.414484,0.000083,34.193547,170.138093,1.522363,28.597740,157.359355,9.977237
986,34.645799,129.410401,100.000000,29.900080,148.791184,85.624949,29.172002,153.048535,12.811359,6.333335,...,11.047440,29.883017,172.375801,0.000083,34.202578,170.098442,1.523092,27.835152,156.350980,10.617848


In [4]:
display(temp)

,right hip-x,right hip-y,right hip-z,right knee-x,right knee-y,right knee-z,right ankle-x,right ankle-y,right ankle-z,left hip-x,...,left heel-z,right toe 1-x,right toe 1-y,right toe 1-z,right toe 2-x,right toe 2-y,right toe 2-z,right heel-x,right heel-y,right heel-z
frame_number,,,,,,,,,,,,,,,,,,,,,
0,39.449136,162.131598,100.184257,36.522369,143.974399,87.395076,34.058394,104.701142,26.561741,20.414385,...,26.447676,34.734083,148.392273,22.461395,38.667728,143.412639,21.829892,32.019455,139.504274,28.874307
1,39.453248,162.145605,100.192216,36.525768,143.985215,87.403087,34.926049,104.691449,26.578776,19.565527,...,26.439908,34.495741,116.735102,18.592852,38.969747,115.313934,19.548701,33.300731,108.728464,23.897386
2,39.777655,164.810217,99.938657,36.855551,146.956253,86.765595,35.172231,105.483416,26.752898,20.601070,...,25.791256,31.941744,116.303816,19.569412,37.405932,116.174572,19.671773,34.452235,109.563609,24.045646
3,40.576205,164.318651,99.697618,36.720703,144.624022,87.810100,35.076347,105.204018,26.674837,21.426205,...,23.986813,32.810106,117.280771,18.661366,38.263885,117.211643,18.723840,34.406710,110.549174,23.129102
4,40.814521,165.856493,99.703791,37.859332,146.285743,87.614330,35.283373,105.706628,26.815713,20.673328,...,26.676634,34.783622,117.879816,18.743699,39.390810,117.722684,18.861933,34.491380,109.765016,24.122696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,33.257117,130.605967,100.000000,30.563866,147.224373,88.762443,31.441949,155.152298,13.520264,7.069172,...,41.828280,31.471683,175.545710,0.000083,35.720899,172.229849,2.211375,30.152580,159.485252,10.643699
985,33.964744,130.588627,100.000000,30.616590,148.772599,85.665044,29.207571,153.064447,12.826504,7.072159,...,39.712779,29.902723,172.414484,0.000083,34.193547,170.138093,1.522363,28.597740,157.359355,9.977237
986,34.645799,129.410401,100.000000,29.900080,148.791184,85.624949,29.172002,153.048535,12.811359,6.333335,...,11.047440,29.883017,172.375801,0.000083,34.202578,170.098442,1.523092,27.835152,156.350980,10.617848


In [77]:
#Do we need the missing frame information 

trueHSR_list = [int(a) for a in trueHSR.split(',')]
trueHSR_indices = [i for i, val in enumerate(temp.index) if val in trueHSR_list] 
fig= plt.figure(figsize = (8.5, 3))
ax1 = fig.add_subplot(111)
ax1.plot(temp.index.values, temp['right heel-z'].values, color = 'b', ls='solid', marker='*', \
        markerfacecolor = 'r', markeredgecolor = 'r', ms = 10, markevery= trueHSR_indices)
plt.xticks(temp.index[0::40], fontsize = 8)
plt.show()

fig= plt.figure(figsize = (8.5, 3))
ax2 = fig.add_subplot(111)
temp_missing_frames_included = temp.reindex(range(temp.index[-1]))
trueHSR_indices_missing_included = [i for i, val in enumerate(temp_missing_frames_included.index) if val in trueHSR_list] 
ax2.plot(temp_missing_frames_included.index, temp_missing_frames_included['right heel-z'].values, color = 'b', ls='solid', marker='*', \
        markerfacecolor = 'r', markeredgecolor = 'r', ms = 10, markevery= trueHSR_indices_missing_included)

plt.xticks(temp.index[0::40], fontsize = 8)
plt.show()




append = np.append(ax.get_xticks(), trueHSR_list)
# ax.set_xticks(trueHSR_list)
# ax.set_xticks(temp.index)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
[i for i, val in enumerate(temp.index) if val in trueHSR_list] 

[35,
 71,
 96,
 166,
 197,
 228,
 316,
 346,
 371,
 401,
 438,
 472,
 504,
 537,
 569,
 627,
 656,
 687,
 716,
 741,
 771,
 799,
 825,
 855,
 885]

### Downsample with smoothing to define fixed shape input tensor for models

In [ ]:
#Use mean with disjoint windows to downsample while smoothing 
#Make sure to preserve count of frames in a frame before smoothing to add as a feature 

